In [52]:
from nltk.tokenize import RegexpTokenizer
from re import sub
from pandas import DataFrame
from pymystem3 import Mystem
import texterra
from itertools import chain

API_KEY = '9988cfb979b80264baeba1386cc7e455f99f943c'

m = Mystem()
t = texterra.API(API_KEY)
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')

# Домашнее задание 1 [10 баллов] 
# До 1.12.17 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



## Кластеризация новостей

Входная коллекция данных состоит из двух частей:
1. events.csv – список 28 резонансных событий первой половины 2017 года, каждому событию присвоен свой порядковый номер (id)
2. raw_news.csv – тексты новостей из различных новостных источников, известно, к какому резонансному событию относится каждая новость (столбец event_id).

Будем считать, что одно событие – это один кластер. В этом домашнем задании вам предстоит:
1. провести кластеризацию текстов новостей и проверить, получается ли восстановить кластерную структуру 
2. проверить, можно ли использовать кластерный анализ для обобщения: найти небольшое число кластеров и проверить, получается ли выделить общие направлени новостей

In [5]:
df_events = DataFrame.from_csv('events.csv')
df_news = DataFrame.from_csv('raw_news.csv')

In [8]:
df_news

,event_id,text
0,1,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...
1,1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П..."
2,1,"Аргументы и Факты (aif.ru), Москва, 14 января..."
3,1,"Google Новости ТОП, Москва, 14 января 2017 АК..."
4,1,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО..."
5,1,"Top Real Estate (topre.ru), Москва, 13 января..."
6,1,"Findnews.ru, Москва, 13 января 2017 ПОЛИЦИЯ Н..."
7,1,"ПРАВДА.info (pravda.info), Москва, 13 января ..."
8,1,"Голос Америки (golos-ameriki.ru), Москва, 13 ..."
9,1,"Выбор Народа (vybor-naroda.org), Москва, 13 я..."


### Часть 1 [2 балла] Предварительная обработка текстов
Проведите предобработку новостей: токенизацию, приведение к нижнему регистру, лемматизацию. Проверьте, есть ли в коллекции дубликаты. Посчитайте, сколько новостей относится к каждому резонансному событию. 

In [ ]:
Для начала смерджим все тексты новостей для того, чтобы получить один единый корпус, с которым

In [168]:
texts = ' '.join(df_news.text.values)

In [166]:
def remove_url(text):
    return sub(r'http\S+', '', text)

In [169]:
texts = remove_url(texts)

In [172]:
tokens = alpha_tokenizer.tokenize(texts)

In [174]:
lemmas = m.lemmatize(texts)
norm_texts = ''.join(lemmas)

KeyboardInterrupt: 

In [ ]:
norm_texts[:100]

In [11]:
df_news.event_id.value_counts()

17    102
28    100
18    100
7     100
10    100
12    100
27    100
16    100
1     100
25    100
26    100
21    100
23    100
3      84
22     82
9      82
24     62
4      62
2      51
11     49
19     45
6      41
8      27
13     24
20      8
15      7
5       2
14      2
Name: event_id, dtype: int64

### Часть 2 [3 балла] Кластеризация текстов
Любым известным вам алгоритмом найдите в коллекции новостей 28 кластеров. Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте любые известные вам меры качества для оценки качества кластеризации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество кластеризации. 

### Часть 3 [5 баллов] Кластеризация текстов (продолжение)
Задайте число кластеров заведомо меньше 28 (например, 5 кластеров). Повторно кластеризуйте новости. Можете ли вы проинтепретировать полученные кластеры? Получается ли с помощью кластерного анализа определить в каких сферах жизни (например, культура, политика, спорт) произошли события?